In [8]:
# Import necessary libraries

import pandas as pd # For data manipulation and analysis
import numpy as np # For numerical operations
import pickle # For serializing and deserializing Python objects
from modelling_utils import train_knn_classifier, evaluate_model, plot_model_comparison, extract_model_features, compute_knn_indices, compute_overlap_matrix, plot_overlap_matrix, split_shuffle_data


In [9]:
# Load features from a pickle file
feature_dict_path = "/home/suraj/Repositories/FM-extractors-radiomics/evaluation/features/dlcs.pkl" # Path to the pickle file containing features
with open(feature_dict_path, 'rb') as file: # Open the file in read binary mode
    data = pickle.load(file) # Load the data from the pickle file

In [10]:
# Store test accuracies for each model
test_accuracies_dict = {} # Initialize an empty dictionary to store test accuracies

# Iterate through each model's features
for model_name, values in data.items(): # Loop through each model and its corresponding features
    # Skip MedImageInsightExtractor
    if model_name == "MedImageInsightExtractor": # Skip the MedImageInsightExtractor model
        continue

    # Extract paths and labels for train, val, and test sets
    train_labels = [v["row"]["Malignant_lbl"] for v in values["train"]] # Extract malignancy labels for the training set
    val_labels = [v["row"]["Malignant_lbl"] for v in values["val"]] # Extract malignancy labels for the validation set
    
    # Stack features
    train_items = np.vstack([v["feature"] for v in values["train"]]) # Stack features for the training set
    val_items = np.vstack([v["feature"] for v in values["val"]]) # Stack features for the validation set
    
    # Concatenate train and val
    all_items = np.vstack((train_items, val_items))
    all_labels = train_labels + val_labels

    # Average across multiple shuffle splits
    n_splits = 5
    split_scores = []
    
    for split in range(n_splits):
        # Get stratified indices for new train/val split with different random seeds
        train_items, train_labels, val_items, val_labels, test_items, test_labels = split_shuffle_data(
            all_items, all_labels, train_ratio=0.5, val_ratio=0.1, random_seed=10+split
        )

        # Train model with hyperparameter optimization
        best_model, study = train_knn_classifier(train_items, train_labels, val_items, val_labels)
        
        # Get score for this split
        split_score = evaluate_model(best_model, test_items, test_labels)
        split_scores.append(split_score)
    
    # Average the scores across splits
    avg_score = np.mean(split_scores)
    
    # Evaluate on test set
    test_accuracies_dict[model_name] = avg_score

[I 2025-02-25 17:08:31,770] A new study created in memory with name: no-name-c510f5e4-4a18-4c15-9f98-30df3df91937
[I 2025-02-25 17:08:31,807] Trial 0 finished with value: 0.6864612188365651 and parameters: {'k': 29}. Best is trial 0 with value: 0.6864612188365651.
[I 2025-02-25 17:08:31,830] Trial 1 finished with value: 0.6018005540166206 and parameters: {'k': 12}. Best is trial 0 with value: 0.6864612188365651.
[I 2025-02-25 17:08:31,850] Trial 2 finished with value: 0.6016274238227146 and parameters: {'k': 11}. Best is trial 0 with value: 0.6864612188365651.
[I 2025-02-25 17:08:31,871] Trial 3 finished with value: 0.6681094182825484 and parameters: {'k': 42}. Best is trial 0 with value: 0.6864612188365651.
[I 2025-02-25 17:08:31,890] Trial 4 finished with value: 0.5832756232686981 and parameters: {'k': 3}. Best is trial 0 with value: 0.6864612188365651.
[I 2025-02-25 17:08:31,910] Trial 5 finished with value: 0.6923476454293628 and parameters: {'k': 28}. Best is trial 5 with value: 0

[I 2025-02-25 17:08:32,462] Trial 29 finished with value: 0.7056786703601108 and parameters: {'k': 26}. Best is trial 29 with value: 0.7056786703601108.
[I 2025-02-25 17:08:32,487] Trial 30 finished with value: 0.5837950138504155 and parameters: {'k': 6}. Best is trial 29 with value: 0.7056786703601108.
[I 2025-02-25 17:08:32,522] Trial 31 finished with value: 0.6376385041551247 and parameters: {'k': 18}. Best is trial 29 with value: 0.7056786703601108.
[I 2025-02-25 17:08:32,550] Trial 32 finished with value: 0.6755540166204985 and parameters: {'k': 41}. Best is trial 29 with value: 0.7056786703601108.
[I 2025-02-25 17:08:32,583] Trial 33 finished with value: 0.6909626038781164 and parameters: {'k': 50}. Best is trial 29 with value: 0.7056786703601108.
[I 2025-02-25 17:08:32,610] Trial 34 finished with value: 0.566308864265928 and parameters: {'k': 2}. Best is trial 29 with value: 0.7056786703601108.
[I 2025-02-25 17:08:32,641] Trial 35 finished with value: 0.6289819944598338 and para

In [11]:
# Plot test accuracies
fig = plot_model_comparison(test_accuracies_dict)
fig.show() # Show the plot

In [12]:
model_features = extract_model_features(data)
model_neighbors = compute_knn_indices(model_features, num_neighbors=10, metric="cosine")
overlap_matrix, model_list = compute_overlap_matrix(model_neighbors)
fig = plot_overlap_matrix(overlap_matrix, model_list)
fig.show()